In [191]:
from google.cloud import bigquery
import numpy as np
import pandas as pd
from unidecode import unidecode
import seaborn as sns
from IPython.display import display, HTML

pd.options.display.max_rows = 5000

In [93]:
client = bigquery.Client()

In [94]:
sql = """
-- count(message) c, 
SELECT message FROM `hackathon-09.dadosBrutos.whatsappRawMessages`
--where length(message) > 10 and message not like '#%'
--group by message
--order by count(message) desc

"""

df = client.query(sql).to_dataframe()
df.head(5)

,message
0,Boa noite\nRastreando meu pedido de número 869...
1,"Oi, boa noite.\nMe chamo Alessandra, estou esp..."
2,"Oi, boa noite.\nMe chamo Alessandra, estou esp..."
3,*Seja bem-vindo(a) a Clínica Veterinária Reino...
4,ACHE UMA BATATA FRITA NO MEIO DAS PIPOCAS \n\n...


In [95]:
df[df.message.str.len() > 10]

,message
0,Boa noite\nRastreando meu pedido de número 869...
1,"Oi, boa noite.\nMe chamo Alessandra, estou esp..."
2,"Oi, boa noite.\nMe chamo Alessandra, estou esp..."
3,*Seja bem-vindo(a) a Clínica Veterinária Reino...
4,ACHE UMA BATATA FRITA NO MEIO DAS PIPOCAS \n\n...
...,...
56733,Boa tarde.\n\nEstou com problema no pedido 744...
56734,Bom dia a respeito do meu pedido 563641104 aca...
56735,"Olá boa tarde. Fiz um pedido de troca, mas não..."
56736,"Olá, Carlos Caja\n \nEsse número de telefone n..."


In [218]:
STOP_WORDS = pd.Series('de a o que e do da em um para é com não uma os no se na por mais as dos como mas foi ao ele das tem à seu sua ou ser quando muito há nos já está eu também só pelo pela até isso ela entre era depois sem mesmo aos ter seus quem nas me esse eles estão você tinha foram essa num nem suas meu às minha têm numa pelos elas havia seja qual será nós tenho lhe deles essas esses pelas este fosse dele tu te vocês vos lhes meus minhas teu tua teus tuas nosso nossa nossos nossas dela delas esta estes estas aquele aquela aqueles aquelas isto aquilo estou está estamos estão estive esteve estivemos estiveram estava estávamos estavam estivera estivéramos esteja estejamos estejam estivesse estivéssemos estivessem estiver estivermos estiverem hei há havemos hão houve houvemos houveram houvera houvéramos haja hajamos hajam houvesse houvéssemos houvessem houver houvermos houverem houverei houverá houveremos houverão houveria houveríamos houveriam sou somos são era éramos eram fui foi fomos foram fora fôramos seja sejamos sejam fosse fôssemos fossem for formos forem serei será seremos serão seria seríamos seriam tenho tem temos tém tinha tínhamos tinham tive teve tivemos tiveram tivera tivéramos tenha tenhamos tenham tivesse tivéssemos tivessem tiver tivermos tiverem terei terá teremos terão teria teríamos teriam'.split())\
    .map(unidecode)

MAX_LEN = 5000

msg_list = pd.DataFrame([s.message.replace('\n', ' ').lower()[:MAX_LEN] for _, s in df.iterrows()], columns=['message'])
word_list = pd.DataFrame([(word.lower()[:MAX_LEN]) for _, s in df.iterrows() for word in s.message.replace('\n', ' ').split(' ')], columns=['words'])
word_list = word_list[~word_list.words.map(unidecode).isin(STOP_WORDS)]

msg_count = pd.Series(msg_list.message.value_counts())
word_count = pd.Series(word_list.words.value_counts())

In [221]:
msg_count.head(10)

#meupedido     16978
meu pedido      1430
#meusaldo        786
boa tarde        763
#minhatroca      744
bom dia          665
oi               593
boa noite        506
#meu pedido      435
olá              315
Name: message, dtype: int64

In [222]:
word_count.head(10)

#meupedido    18264
pedido         5490
               2519
boa            2350
dia            1318
bom            1286
tarde          1085
#meu            903
gostaria        882
#meusaldo       792
Name: words, dtype: int64

In [223]:
msg_count[msg_count.index.str.len() > 5].head(50)

#meupedido                            16978
meu pedido                             1430
#meusaldo                               786
boa tarde                               763
#minhatroca                             744
bom dia                                 665
boa noite                               506
#meu pedido                             435
#minhadevolução                         288
meupedido                               244
#minhadevolucao                         144
gostaria de falar com um atendente      123
#reenviar                               105
#meuspedidos                             98
pedido                                   94
# meu pedido                             91
#pedido                                  77
#meupedidi                               70
minha troca                              69
bom dia!                                 67
olá boa tarde                            65
boa tarde!                               64
# meupedido                     

In [233]:
msg_df = pd.DataFrame(msg_count)
msg_df = msg_df[msg_df.index.map(lambda w: len(w.replace('\n', ' ').split(' ')) > 3)]
msg_df['pedido'] = msg_df.index.map(lambda w: 'pedido' in w)
msg_df['troca'] = msg_df.index.map(lambda w: 'troca' in w or 'devolucao' in unidecode(w))
msg_df['voucher'] = msg_df.index.map(lambda w: 'voucher' in w or 'saldo' in w)
# msg_df = msg_df.sort_values('message')
msg_df

,message,pedido,troca,voucher
gostaria de falar com um atendente,123,False,False,False
"*auxiliar de creche 2019 vagas abertas:* - monitor escolar - merendeira - fiscal de corredor - coord. de ensino - motorista - porteiro - jardineiro - zelador - vigia e mais... *salário a partir de:* r$1.864,26 (mesmo sem experiência). *vagas disponíveis em:* http://bit.ly/vagascrechewp4 *- clique no link acima para agendar sua entrevista.* *início imediato!*",17,False,False,False
meu pedido não chegou,14,True,False,False
não recebi meu pedido,13,True,False,False
meu pedido ainda não chegou,12,True,False,False
...,...,...,...,...
como faço p saber sobres os produtos? não recebi todos,1,False,False,False
número do pedido #879619270,1,True,False,False
"aguardando autorização para devolução: 732543991 solicitado em 05/09/2019 valor da devolução r$ 99,90",1,False,True,False
posso mandar um áudio explicando?,1,False,False,False


In [234]:
msg_df.to_csv('mensagens.csv')

In [202]:
msg_troca = msg_count[msg_count.index.map(lambda w: 'troca' in w)]
msg_troca.head(5000)

#minhatroca                                                                                             744
minha troca                                                                                              69
troca                                                                                                    58
#minha troca                                                                                             35
minhatroca                                                                                               30
#troca                                                                                                   24
#minhatroca-numero da troca                                                                              11
preciso fazer uma troca                                                                                  10
quero fazer uma troca                                                                                     8
como fazer troca            